In [44]:
from tqdm import tqdm

from utils import *

In [45]:
SAMPLE_SIZE = 5000

In [46]:
fix_all()
karenina_str, warandpeace_ru_str, warandpeace_en_str = read_data()
print(f'Anna Karenina length: {len(karenina_str)}')
print(f'War And Peace RU length: {len(warandpeace_ru_str)}')
print(f'War And Peace EN length: {len(warandpeace_en_str)}')

Anna Karenina length: 1719443
War And Peace RU length: 656402
War And Peace EN length: 3112013


## Часть 1 и 2. Декодирование сообщения путем подсчета статистик


Код получился довольно большой, поэтому от помещен в отдельный файл

In [47]:
karenina_freq = get_tokens_freq(karenina_str)
karenina_sample = karenina_str[:SAMPLE_SIZE]
encoded_text, decoded_text, score = compare_encodings(karenina_str, karenina_sample)
print(f'Score: {score}')

Score: 0.2734


In [48]:
karenina_sample[:100]

'анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все'

In [49]:
decoded_text[:100]

'иееи диваенеи омне ны тикуч ыеикаенсуч вокиеол рзли сортсожо еийнеиастг тсилюаь ифовныкок фвиыоь лта'

In [50]:
karenina_tokens = get_ngrams(karenina_str, n=2)
karenina_freq = get_tokens_freq(karenina_tokens).keys()
karenina_sample = karenina_str[:SAMPLE_SIZE]
encoded_text, decoded_text, score = compare_encodings(karenina_str, karenina_sample, n=2)
print(f'Score: {score}')

Score: 0.0524


In [51]:
karenina_sample[:100]

'анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все'

In [52]:
decoded_text[:100]

'новероащго у сг  доты з вон  килто з валетцого у сц овийтибыким етрымалю  дуав цсхвая т лсусисвзизся'

Из результатов выше видно, что просто метод декодирования через подсчитывание частот не очень хорошо работает, т.к. только изредка можно с трудом распознать какие-то отдельные слова

In [53]:
warandpeace_en_freq = get_tokens_freq(warandpeace_en_str).keys()
warandpeace_en_sample = warandpeace_en_str[:SAMPLE_SIZE]
encoded_text, decoded_text, score = compare_encodings(warandpeace_en_str, warandpeace_en_sample)
print(f'Score: {score}')

Score: 0.2354


In [54]:
warandpeace_en_sample[:100]

'the project gutenberg ebook of war and peace by leo tolstoy  this ebook is for the use of anyone any'

In [55]:
decoded_text[:100]

'ait snljtha pgatuctnp tcllf lk bon ouy stoht cv wtl alwmalv  aiem tcllf em kln ait gmt lk ouvlut ouv'

In [56]:
warandpeace_en_tokens = get_ngrams(warandpeace_en_str, n=2)
warandpeace_en_freq = get_tokens_freq(warandpeace_en_tokens).keys()
warandpeace_en_sample = warandpeace_en_str[:SAMPLE_SIZE]
encoded_text, decoded_text, score = compare_encodings(warandpeace_en_str, warandpeace_en_sample, n=2)
print(f'Score: {score}')

Score: 0.0912


In [57]:
warandpeace_en_sample[:100]

'the project gutenberg ebook of war and peace by leo tolstoy  this ebook is for the use of anyone any'

In [58]:
decoded_text[:100]

'lleeovce dbll inevosnsi aymatudrivs do yrihobeurcabuadfo asieavet t s anheomprutmoulncsenim h nt oon'

Аналогичный результат получился и для тектсов на английском языке

In [59]:
warandpeace_ru_freq = get_tokens_freq(warandpeace_ru_str).keys()
warandpeace_ru_sample = warandpeace_ru_str[:SAMPLE_SIZE]
encoded_text, decoded_text, score = compare_encodings(warandpeace_ru_str, warandpeace_ru_sample)
print(f'Score: {score}')

Score: 0.5174


In [60]:
warandpeace_ru_sample[:100]

'война и мир  самый известный роман льва николаевича толстого как никакое другое произведение писател'

In [61]:
decoded_text[:100]

'сойне и див  ледгй иьсалтнгй воден рзсе никореасиые торлтобо кек никекоа мвпбоа увоиьсаманиа уилетар'

In [62]:
warandpeace_ru_tokens = get_ngrams(warandpeace_ru_str, n=2)
warandpeace_ru_freq = get_tokens_freq(warandpeace_ru_tokens).keys()
warandpeace_ru_sample = warandpeace_ru_str[SAMPLE_SIZE:2*SAMPLE_SIZE]
encoded_text, decoded_text, score = compare_encodings(warandpeace_ru_str, warandpeace_ru_sample, n=2)
print(f'Score: {score}')

Score: 0.0534


In [63]:
warandpeace_ru_sample[:100]

'онапарте непобедим и что вся европа ничего не может против него и я не верю ни в одном слове ни гард'

In [64]:
decoded_text[:100]

'начттоекибголяй  яуздьббу ве с лсттьержаущвмраиз шхмев сицолаеяхтыынтаоу как норрори мти в пи ене ря'

В результатах выше для токенов длины 1 получился относительно неплохой результат, но декодирование по токенам длинны 2 стабильно показывает результат хуже, чем декодирование по токенам длины 1. Думаю, что это связано с тем, что вариаций токенов длины 2 больше чем токенов длины 1, и отследить частотные закономерности для таких токенов сложнее

## Часть 3. Построение модели MCMC

In [65]:
SAMPLE_SIZE = 1000

In [103]:
karenina_sample = karenina_str[:1000]
karenina_freq = get_tokens_freq(get_ngrams(karenina_str, 2), dens=True)
encoded_text = encode_text(karenina_sample, n=1)
mcmc_model = MCMCModel(karenina_freq, n=1, encoded_sample=encoded_text)
decoded_text = mcmc_model.fit(encoded_text, 
    n=2, 
    n_iter=10000,
    sample=karenina_sample
)

100%|██████████| 10000/10000 [00:14<00:00, 695.86it/s, Accuracy: 0.41]


In [104]:
mcmc_model.transform(encoded_text)[:100]

'леел млкнесел обсе сж илдря желднестря кодлеов уавл тоуитого елчселнтиь итлвхню лфоксждод фклжою вин'

In [68]:
karenina_sample[:100]

'анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все'

Из результатов выше видно, что получить хорошее декодирование не получилось, хоть результат и лучше того, что получилось в части 1. Тестировал несколько раз, получить accuracy больше 0.5 не вышло

In [111]:
warandpeace_en_sample = warandpeace_en_str[:15000]
warandpeace_en_freq = get_tokens_freq(get_ngrams(warandpeace_en_str, 2), dens=True)
encoded_text = encode_text(warandpeace_en_sample, n=1)
mcmc_model = MCMCModel(warandpeace_en_freq, n=1, encoded_sample=encoded_text)
decoded_text = mcmc_model.fit(encoded_text,
    n=2, 
    n_iter=10000,
    sample=warandpeace_en_sample
)

100%|██████████| 10000/10000 [03:02<00:00, 54.79it/s, Accuracy: 0.44]


In [112]:
mcmc_model.transform(encoded_text)[:100]

'teo nrijoht kgtolwork owiiz iv bar ald noaho wy poi tipftiy  tesf owiiz sf vir teo gfo iv alyilo aly'

In [75]:
warandpeace_en_sample[:100]

'the project gutenberg ebook of war and peace by leo tolstoy  this ebook is for the use of anyone any'

Для примера выше результат получился аналогично результату пиз первого примера, хотя это все еще лучше обычного частотного метода

In [113]:
warandpeace_ru_sample = warandpeace_ru_str[:1000]
warandpeace_ru_freq = get_tokens_freq(get_ngrams(warandpeace_ru_str, 2), dens=True)
encoded_text = encode_text(warandpeace_ru_sample, n=1)
mcmc_model = MCMCModel(warandpeace_ru_freq, n=1, encoded_sample=encoded_text)
decoded_text = mcmc_model.fit(encoded_text,
    n=2, 
    n_iter=10000,
    sample=warandpeace_ru_sample
)

100%|██████████| 10000/10000 [00:14<00:00, 694.30it/s, Accuracy: 0.72]


In [114]:
''.join(decoded_text)[:100]

'нойла и мир  вамый ишневслый ромал дтна ликодаениза содвсобо как ликакое чрубое проишнечелие пивасед'

In [78]:
warandpeace_ru_sample[:100]

'война и мир  самый известный роман льва николаевича толстого как никакое другое произведение писател'

Этот пример получился достаточно удачным. Пришлось несколько раз запускать метод, чтобы получить относительно высокий скорр. Также видно, что многи слова достаточно легко угадываются после декодирования

## Часть 4. Декодирование зашифрованного сообщения

In [80]:
ENCODED_MSG_1 = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [93]:
warandpeace_ru_sample = warandpeace_ru_str[:1000]
warandpeace_ru_freq = get_tokens_freq(get_ngrams(warandpeace_ru_str, 2), dens=True)
mcmc_model = MCMCModel(warandpeace_ru_freq, n=1, encoded_sample=ENCODED_MSG_1)
encoded_text = ENCODED_MSG_1
decoded_text = mcmc_model.fit(encoded_text,
    n=2, 
    n_iter=150000
)

100%|██████████| 150000/150000 [00:51<00:00, 2890.14it/s]


In [94]:
''.join(decoded_text)

'если ву вичите норкальную или помти норкальную тедст ы этого сообщения доторую легдо промитать сдорее всего ву все счелали правильно и полымите кадсикальную балл за послечнее метвертое зачание дырса хотя донемно я нимего не обещаш'

Сообщение выше получилось расшировать достаточно неплохо на мой взгляд. Основная сложность при расшифровывании, да и при выполнении всего задания заключалась в том, что приходилось несколько раз перезапускать алгоритм и ловить удачные сэмплы